In [1]:
import pandas as pd

In [7]:
info = pd.read_csv('SPL/Library_Collection_Inventory.csv') #, index_col=0)
info = info[['BibNum', 'ISBN', 'Title', 'Author', 'PublicationYear', 'Subjects', 'ItemType']]
info = info[info['ItemType'].str.contains('bk')]
info['ISBN'] = info['ISBN'].str.split(',', expand=False)
# expand the ISBN column into multiple rows
info = info.explode('ISBN').reset_index(drop=True)
info.dropna(subset=['ISBN'], inplace=True)
info.head(3)

,BibNum,ISBN,Title,Author,PublicationYear,Subjects,ItemType
0,3011076,1481425730,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk
1,3011076,1481425749,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk
2,3011076,9781481425735,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk


In [8]:
#book_set = pd.merge(book_set, info, left_on='isbn10', right_on='ISBN', how='left')
book_set = info.copy()
book_set.dropna(subset=['BibNum'], inplace=True)
book_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4176490 entries, 0 to 4451680
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   BibNum           int64 
 1   ISBN             object
 2   Title            object
 3   Author           object
 4   PublicationYear  object
 5   Subjects         object
 6   ItemType         object
dtypes: int64(1), object(6)
memory usage: 254.9+ MB


In [9]:
prefix = 'Checkouts_By_Title_Data_Lens_' # prefix for file names
suffix = '.csv' # suffix for file names
years = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
for year in years:
    df = pd.read_csv('SPL/' + prefix + str(year) + suffix)
    df = df[['BibNumber']]
    df['Checkouts'] = 1
    df = df.groupby('BibNumber').sum().reset_index()
    df.columns = ['BibNum', 'Checkouts_' + str(year)]
    book_set = pd.merge(book_set, df, on='BibNum', how='left')

/var/folders/73/s_z37x2x4gv93j_kb8744gmh0000gn/T/ipykernel_26415/3039891236.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SPL/' + prefix + str(year) + suffix)


In [13]:
book_set.fillna(0, inplace=True)
book_set['first_pub'] = book_set['PublicationYear'].str.split(',').str[0].str.split('.').str[0].str.split('-').str[0].str.split(' ').str[0]

In [15]:
book_set['first_pub'] = pd.to_numeric(book_set['first_pub'], errors='coerce')

In [16]:
book_set['years since pub'] = 2018 - book_set['first_pub']

In [17]:
book_set['sales since pub'] = book_set['Checkouts_2005'] + book_set['Checkouts_2006'] + book_set['Checkouts_2007'] + book_set['Checkouts_2008'] + book_set['Checkouts_2009'] + book_set['Checkouts_2010'] + book_set['Checkouts_2011'] + book_set['Checkouts_2012'] + book_set['Checkouts_2013'] + book_set['Checkouts_2014'] + book_set['Checkouts_2015'] + book_set['Checkouts_2016'] + book_set['Checkouts_2017']
book_set['avg sales per year'] = book_set['sales since pub'] / book_set['years since pub']

In [18]:
book_set.sort_values(by='avg sales per year', ascending=False, inplace=True) # sort by avg sales per year

In [23]:
book_set = book_set[(book_set['avg sales per year'] > 0) & (book_set['avg sales per year'] < 100000)] # remove books with no sales
book_set = book_set.drop_duplicates(subset=['ISBN']) # remove duplicates
subset = book_set[['ISBN', 'Title', 'Author', 'PublicationYear', 'Subjects', 'ItemType', 'avg sales per year']][:10000]

In [24]:
subset

,ISBN,Title,Author,PublicationYear,Subjects,ItemType,avg sales per year
2556759,1594633665,The girl on the train / Paula Hawkins.,"Hawkins, Paula",2015.,"Railroad travel Fiction, Witnesses Fiction, Mu...",acbk,1725.000000
2705209,1594634122,The girl on the train / Paula Hawkins.,"Hawkins, Paula",2015.,"Railroad travel Fiction, Witnesses Fiction, Mu...",acbk,1725.000000
2807755,9781594634123,The girl on the train / Paula Hawkins.,"Hawkins, Paula",2015.,"Railroad travel Fiction, Witnesses Fiction, Mu...",acbk,1725.000000
2705210,9781594633669,The girl on the train / Paula Hawkins.,"Hawkins, Paula",2015.,"Railroad travel Fiction, Witnesses Fiction, Mu...",acbk,1725.000000
731043,0735211205,Into the water / Paula Hawkins.,"Hawkins, Paula",2017.,"Murder Fiction, Rivers Fiction, Women Crimes a...",acbk,1646.000000
...,...,...,...,...,...,...,...
3769409,0803734085,Betty Bunny wants everything / written by Mich...,"Kaplan, Michael B.",2012.,"Behavior Fiction, Greed Fiction, Family life F...",jcbk,99.500000
2644084,9781594489785,The ten-year nap / Meg Wolitzer.,"Wolitzer, Meg",2008.,"Women Fiction, Unemployment Fiction, Motherhoo...",acbk,99.500000
1847583,1455527955,Someone always knows / Marcia Muller.,"Muller, Marcia",2016.,"McCone Sharon Fictitious character Fiction, Wo...",acbk,99.500000
22444,0060583606,The Berenstain Bears' family reunion / by Stan...,"Berenstain, Stan, 1923-2005",2009.,"Stories in rhyme Juvenile fiction, Family reun...",jcbk,99.444444


In [26]:
subset.to_csv('Exported/SPL_Sel.csv')